## **This code aims to loop over combinations of GRU models and data and store results**

In [2]:
import pandas as pd
import numpy as np
import os
import time
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU 0 in first notebook

In [3]:
project_dir = "/home/jupyter-tfg2425paula/prediction_project_v3"
os.chdir(project_dir)

clean_data_dir = os.path.join(project_dir, "00_data/clean")
horizontal_data_dir = os.path.join(project_dir, "00_data/horizontal_structure")
results_dir = os.path.join(project_dir, "02_results")
pca_data_dir = os.path.join(project_dir, "00_data/pca")

### **GRU Model**

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class GRU3DClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(GRU3DClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :]) 
        # return self.sigmoid(out)
        return out

### **LSTM Model**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class StockPriceLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1, dropout=0.0):
        super(StockPriceLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, 
                            batch_first=True, dropout=dropout)
    
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        batch_size = x.size(0)  # Get the batch size dynamically

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)  # (num_layers, batch_size, hidden_dim)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)  # (num_layers, batch_size, hidden_dim)
        
        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :]) 
        # out = self.sigmoid(out)
        return out
    

### **Set folders**

Macro Folders

In [6]:
model_type = "lstm"

Processing

In [7]:
processing = ["clean", "pca"]
processing = ["clean"]

Folders

In [8]:
stocks = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'SPX']
stocks = ['AAPL']
types_securities = ["single_name", "options", "technical"]
types_securities = ["options"]

Different files

In [9]:
years = ["15y", "10y", "5y", "2y"]
years = ["15y"]
window_sizes = [5, 10, 50, 100]
window_sizes = [5]
train_sizes = [80, 90, 95]
train_sizes = [95]

Same file

In [10]:
thresholds = [0.3, 0.35, 0.4, 0.45, 0.5]
thresholds = [0.5]
learning_rates = [0.005, 0.008, 0.009, 0.01]
learning_rates = [0.01]
num_epochs_list = [100, 200]
num_epochs_list = [100]
batch_sizes = [16, 32]
batch_sizes = [16]
prediction_thresholds = [0.35, 0.4, 0.45, 0.5]
prediction_thresholds = [0.5]

#### **Hyperparameters**

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hidden_size = 64  
output_size = 2  
num_layers = 2
dropout = 0.2

criterion = nn.CrossEntropyLoss()

#### **Last data modifications**

In [12]:
def reshape_remove_characters(df):

    X = np.array([np.stack(row) for row in df.drop(columns=['Target']).values])
    y = df['Target'].values

    smote = SMOTE(random_state=42)
    n_samples, timesteps, n_features = X.shape
    X_flat = X.reshape((n_samples, timesteps * n_features))
    X_flat = np.where(X_flat == 'ç', 0, X_flat)

    X_resampled = X_flat.reshape((-1, timesteps, n_features))
    
    return X_resampled, y

### **Evaluation function**

In [13]:
def evaluate_rolling_unchanged_model_threshold(
    model, 
    X, 
    y, 
    criterion, 
    optimizer, 
    device, 
    train_size, 
    batch_size, 
    num_epochs, 
    lower_threshold
):
    """
    Evaluate a PyTorch model using a rolling prediction approach for time series,
    training the model only once on the initial training set. For each time step
    after train_size, the model makes a prediction without further parameter updates.
    Only predicts +1 or -1 if the probability of class 1 is above/below given thresholds;
    otherwise, predicts 0. Accuracy is computed only on nonzero predictions.

    Args:
        model:          PyTorch model to evaluate.
        X:              Feature data (numpy array).
        y:              Target data (numpy array).
        criterion:      Loss function (e.g., CrossEntropyLoss).
        optimizer:      Optimizer (e.g., Adam).
        device:         Device for computation (CPU or GPU).
        train_size:     Initial size of the training data (int or float).
                        If < 1, treated as fraction of total length.
        batch_size:     Batch size for training.
        num_epochs:     Number of epochs for initial training only.
        lower_threshold: Probability threshold below which model predicts -1.
        upper_threshold: Probability threshold above which model predicts +1.

    Returns:
        dict: Dictionary with the following keys:
            - "rolling_predictions": All predictions (-1, 0, +1) across the test period.
            - "rolling_targets": Corresponding true targets in [-1, +1].
            - "filtered_predictions": Nonzero predictions only.
            - "filtered_targets": Targets corresponding to nonzero predictions.
            - "accuracy_nonzero": Accuracy computed only on nonzero predictions.
    """

    # Convert X, y to tensors
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)

    # Determine initial training set size
    if train_size < 1.0:
        lower_bound = int(train_size * len(X))
    else:
        lower_bound = train_size

    # -------------------------
    # 1) SINGLE TRAINING PHASE
    # -------------------------
    model.to(device)
    model.train()
    
    X_train = X[:lower_bound].to(device)
    y_train = y[:lower_bound].to(device)

    train_dataset = TensorDataset(X_train, y_train)
    trainloader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=False,         # Keep False if order matters; True for better generalization
        # num_workers=4,         # Adjust based on your CPU cores
        # pin_memory=True,       # Speeds up transfer if using GPUs
        drop_last=False        # Ensure the last batch is included
    )

    epoch_train_losses = []
    for epoch in range(num_epochs):
        # torch.cuda.empty_cache()
        epoch_loss = 0.0
        for X_batch, y_batch in trainloader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            pred_y = model(X_batch)   # [batch_size, num_classes]
            loss = criterion(pred_y, y_batch)
            loss.backward()

            # Gradient clipping (optional)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += loss.item()
               
        if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1:
            print(f"[Train] Epoch {epoch+1}/{num_epochs}, Loss={epoch_loss/len(trainloader):.4f}")

        epoch_train_losses.append(epoch_loss/len(trainloader))
        
    loss_decrease_percentage = ((epoch_train_losses[-1] - epoch_train_losses[0]) / epoch_train_losses[0]) * 100
    # ---------------------------------
    # 2) ROLLING PREDICTIONS, NO UPDATE
    # ---------------------------------
    model.eval()

    rolling_predictions = []
    rolling_targets     = []

    for i in range(lower_bound, len(X)):
        # Single-step "test" sample
        X_test = X[i:i+1].to(device)  # shape: (1, num_features)
        y_test = y[i:i+1].to(device)  # shape: (1, )

        with torch.no_grad():
            # Forward pass
            pred_y = model(X_test)  # [1, num_classes]
            probabilities = torch.softmax(pred_y, dim=1).cpu().numpy()  # shape: (1, 2)
            prob_class_1  = probabilities[:, 1]  # shape: (1,)

            # Threshold-based logic
            # Initialize all predictions to 0
            pred_classes = np.zeros_like(prob_class_1)
            # Predict -1 if prob < lower_threshold
            pred_classes[prob_class_1 < lower_threshold] = -1
            # Predict +1 if prob > upper_threshold
            pred_classes[prob_class_1 > 1-lower_threshold] = 1

        rolling_predictions.append(pred_classes[0])  # scalar
        rolling_targets.append(y_test.item())

    rolling_predictions = np.array(rolling_predictions)
    rolling_targets = np.array(rolling_targets).astype(int)

    # Convert any 0-labeled targets to -1 if your original data is in [-1, +1]
    # (Sometimes y might be {0,1} or {-1, +1}; adapt as needed.)
    rolling_targets[rolling_targets == 0] = -1

    # Filter out zero predictions
    nonzero_mask = rolling_predictions != 0
    filtered_preds = rolling_predictions[nonzero_mask]
    filtered_targets = rolling_targets[nonzero_mask]

    if len(filtered_preds) == 0:
        accuracy_nonzero = None
        print("No nonzero predictions, cannot compute thresholded accuracy.")
    else:
        accuracy_nonzero = accuracy_score(filtered_targets, filtered_preds)
        print(f"Accuracy on Nonzero Predictions: {accuracy_nonzero:.4f}")

    return {
        "rolling_predictions": rolling_predictions,
        "rolling_targets": rolling_targets,
        "filtered_predictions": filtered_preds,
        "filtered_targets": filtered_targets,
        "accuracy_nonzero": accuracy_nonzero,
        "loss_decrease_percentage": loss_decrease_percentage
    }

### **Run the loop**

In [14]:
# folder
results_list = []
for stock in stocks:
    for security_type in types_securities:
        output_folder = os.path.join(results_dir, f"{model_type}/{stock}/{security_type}") 
        os.makedirs(output_folder, exist_ok=True)
        # files
        for period in years:
            # load original data as well (for info purposes)
            filename = f"{security_type}/{stock}/{period}_data.csv"
            original_input_filepath = os.path.join(clean_data_dir, filename)
            original_data = pd.read_csv(original_input_filepath)
            start_date = original_data.loc[0, "Date"]
            end_date = original_data.iloc[-1]["Date"]
            
            for possible_train_size in train_sizes:
                
                results_csv_path = os.path.join(output_folder, f"{period}_{possible_train_size}.csv")
                
                # columns, same file
                for window_size in window_sizes:
                    print(f"{stock}, {security_type}, {period}, {possible_train_size}, {window_size}")
                    
                    # load data
                    pkl_filename = f"clean/{security_type}/{stock}/{period}_{window_size}_data.pkl"
                    input_filepath = os.path.join(horizontal_data_dir, pkl_filename)
                    print(input_filepath)
                    input_df = pd.read_pickle(input_filepath)
                    
                    X_resampled, y_resampled = reshape_remove_characters(input_df)

                    input_size = X_resampled.shape[2]
                    train_size = int(X_resampled.shape[0]*possible_train_size/100)
                    test_size = X_resampled.shape[0] - train_size

                    # generate model
                    model = StockPriceLSTM(input_size, hidden_size, output_size)
                    # model = GRU3DClassifier(input_size, hidden_size, output_size, num_layers, dropout)
                    # model = torch.nn.DataParallel(model)
                    model = model.to(device)
                    
                    for learning_rate in learning_rates:
                        
                        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
                        
                        for num_epochs in num_epochs_list:
                            for prediction_threshold in prediction_thresholds:
                                for batch_size in batch_sizes:

                                    print(f"Training {stock} | LR: {learning_rate} | Epochs: {num_epochs} | Batch: {batch_size} | Prediction Threshold: {prediction_threshold}")
                                    
                                    start_time = time.time()

                                    result = evaluate_rolling_unchanged_model_threshold(
                                        model, X_resampled, y_resampled, criterion, 
                                                                   optimizer, device, train_size, batch_size, num_epochs, lower_threshold=prediction_threshold)     

                                    rolling_predictions = result["rolling_predictions"]
                                    rolling_targets = result["rolling_targets"]
                                    test_accuracy = result["accuracy_nonzero"]
                                    loss_decrease_percentage = result["loss_decrease_percentage"]
                                    nonzero_preds = np.count_nonzero(result["rolling_predictions"])


                                    end_time = time.time()    
                                    execution_time = end_time - start_time
                                    
                                    # --------------------------------------------
                                    # 1) Create a record (dictionary) for this run
                                    # --------------------------------------------
                                    run_record = {
                                        "start_date": start_date,
                                        "end_date": end_date,
                                        "execution_time": execution_time,
                                        "test_size": test_size,
                                        "nonzero_preds": nonzero_preds,
                                        "accuracy": test_accuracy,
                                        "prediction_threshold": prediction_threshold,
                                        
                                        "window_size": window_size,
                                        "learning_rate": learning_rate,
                                        "num_epochs": num_epochs,
                                        "train_loss_change_pctg": loss_decrease_percentage,
                                        
                                        "batch_size": batch_size,
                                        
                                        "output_size": output_size,
                                        "hidden_size": hidden_size,
                                        "num_layers": num_layers,
                                        "dropout_rate": dropout,
                                        "optimizer": optimizer.__class__.__name__,
                                        "criterion": criterion
                                        
                                    }
                                    
                                    # --------------------------------------------
                                    # 2) Append the dictionary to the results list
                                    # --------------------------------------------
                                    results_list.append(run_record)
                
                # ----------------------------------------------------------------
                # 3) Write to CSV *once* after all window_sizes for this setup
                # ----------------------------------------------------------------
                if len(results_list) > 0:
                    df = pd.DataFrame(results_list)
                    
                    # If CSV already exists, append without header
                    if os.path.exists(results_csv_path):
                        df.to_csv(results_csv_path, mode='a', header=False, index=False)
                    else:
                        df.to_csv(results_csv_path, index=False)
                    
                    # Clear results_list to avoid duplication
                    results_list = []
                                    

AAPL, options, 15y, 95, 5
/home/jupyter-tfg2425paula/prediction_project_v3/00_data/horizontal_structure/clean/options/AAPL/15y_5_data.pkl
Training AAPL | LR: 0.01 | Epochs: 100 | Batch: 16 | Prediction Threshold: 0.5
[Train] Epoch 5/100, Loss=0.6923
[Train] Epoch 10/100, Loss=0.6754
[Train] Epoch 15/100, Loss=0.6125
[Train] Epoch 20/100, Loss=0.5152
[Train] Epoch 25/100, Loss=0.4226
[Train] Epoch 30/100, Loss=0.3315
[Train] Epoch 35/100, Loss=0.2615
[Train] Epoch 40/100, Loss=0.2094
[Train] Epoch 45/100, Loss=0.1780
[Train] Epoch 50/100, Loss=0.1432
[Train] Epoch 55/100, Loss=0.1050
[Train] Epoch 60/100, Loss=0.1082
[Train] Epoch 65/100, Loss=0.1127
[Train] Epoch 70/100, Loss=0.1077
[Train] Epoch 75/100, Loss=0.0826
[Train] Epoch 80/100, Loss=0.0970
[Train] Epoch 85/100, Loss=0.0864
[Train] Epoch 90/100, Loss=0.0659
[Train] Epoch 95/100, Loss=0.0648
[Train] Epoch 100/100, Loss=0.1268
Accuracy on Nonzero Predictions: 0.5408


In [10]:
# ONE EXAMPLE
security_type = "technical"
stock = "AAPL"
period = "15y"
window_size = 5

pkl_filename = f"pca/{security_type}/{stock}/{period}_{window_size}_data.pkl"
input_filepath = os.path.join(horizontal_data_dir, pkl_filename)
input_df = pd.read_pickle(input_filepath)
                    
input_df               

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter-tfg2425paula/prediction_project_v3/00_data/horizontal_structure/pca/technical/AAPL/15y_5_data.pkl'

In [16]:
input_df.columns

Index(['Adj Close', 'High', 'Low', 'Open', 'Volume', 'volume_adi',
       'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em',
       'volume_vpt', 'volume_vwap', 'volume_mfi', 'volume_nvi',
       'volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbw',
       'volatility_bbp', 'volatility_bbhi', 'volatility_bbli',
       'volatility_kcc', 'volatility_kch', 'volatility_kcl', 'volatility_kcw',
       'volatility_kcp', 'volatility_kchi', 'volatility_kcli',
       'volatility_dcl', 'volatility_dch', 'volatility_dcm', 'volatility_dcw',
       'volatility_dcp', 'volatility_atr', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv',
       'trend_ic

In [7]:
security_type = "options"
stock = "AAPL"
period = "10y"
original_input_df = pd.read_csv(os.path.join(pca_data_dir, f"{security_type}/{stock}/{period}_data.csv"))
# original_input_df = original_input_df.drop(columns = ["Date"])
original_input_df

,Rotated_PC1,Rotated_PC2,Rotated_PC3,Rotated_PC4,Rotated_PC5,Rotated_PC6,Target
0,-0.668148,1.466436,-0.305846,0.854463,1.170655,-0.782029,0.0
1,-0.673351,1.534179,-0.300122,1.235940,-0.639577,-0.822672,1.0
2,-0.705821,1.609841,-0.318937,0.153327,0.728884,-0.756895,0.0
3,-0.738629,1.600690,-0.314730,-0.184954,-0.063855,-0.739186,0.0
4,-0.728968,1.605263,-0.251254,-0.523637,-0.103699,-0.724045,0.0
...,...,...,...,...,...,...,...
2605,-0.423556,-0.003686,-0.388047,-0.485901,0.833111,0.297979,1.0
2606,-0.485159,0.014004,-0.345136,-0.719921,0.015915,0.282589,1.0
2607,-0.296495,0.035988,-0.379711,-0.731220,0.146390,-0.010709,0.0
2608,-0.489598,0.054799,-0.364703,-0.712787,-0.198555,0.321537,1.0


In [37]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaling_method = "standard"
selected_scale_cols = list(original_input_df.drop(columns=["Target"]).columns)
scaled_df = scale_data(original_input_df, selected_scale_cols, scaling_method)
scaled_df

/tmp/ipykernel_21431/1067051062.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipykernel_21431/1067051062.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].fillna(df[selected_scale_cols].mean(), inplace=True)


,AAPL_CALL_OM,AAPL_CALL_O1,AAPL_CALL_OY,AAPL_CALL_OI,AAPL_CALL_VM,AAPL_PUT_OM,AAPL_PUT_O1,AAPL_PUT_OY,AAPL_PUT_OI,AAPL_PUT_VM,Return,Target
0,-0.659948,0.023948,1.232190,-1.135737,-0.867662,0.066764,0.076497,1.069240,-1.141176,-1.135861,1.939250,0.0
1,0.032441,-0.019636,1.298093,-1.130776,-0.940435,-0.689031,0.020175,1.024588,-1.132042,-1.164318,-0.534837,0.0
2,0.005087,0.012214,1.306002,-1.125519,-0.960481,-0.697336,0.045282,1.024588,-1.127452,-1.187863,-0.146881,0.0
3,0.005087,0.012214,1.306002,-1.125519,-0.949940,-0.697336,0.045282,1.024588,-1.127452,-1.179098,-0.064682,0.0
4,-0.319737,0.163080,1.541936,-1.115336,-0.939400,-0.448173,0.133498,1.105342,-1.122220,-1.170332,-1.249798,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3910,-0.328285,-0.483409,-0.570931,0.246195,-0.429808,-0.587705,-0.613622,-0.659835,0.338419,-0.142768,0.837548,1.0
3911,-0.345381,-0.435356,-0.621018,0.256808,-0.663735,-0.622587,-0.612265,-0.605682,0.358566,-0.510329,0.011964,1.0
3912,-0.507794,-0.471116,-0.555114,0.264063,-0.674956,-0.486378,-0.595979,-0.583831,0.374753,-0.631355,0.147328,0.0
3913,-0.319737,-0.456589,-0.552478,0.266275,-0.655556,-0.644182,-0.578336,-0.591432,0.392888,-0.387046,-0.205578,1.0


In [42]:
scaled_df[:10]

,AAPL_CALL_OM,AAPL_CALL_O1,AAPL_CALL_OY,AAPL_CALL_OI,AAPL_CALL_VM,AAPL_PUT_OM,AAPL_PUT_O1,AAPL_PUT_OY,AAPL_PUT_OI,AAPL_PUT_VM,Return,Target
0,-0.659948,0.023948,1.232190,-1.135737,-0.867662,0.066764,0.076497,1.069240,-1.141176,-1.135861,1.939250,0.0
1,0.032441,-0.019636,1.298093,-1.130776,-0.940435,-0.689031,0.020175,1.024588,-1.132042,-1.164318,-0.534837,0.0
2,0.005087,0.012214,1.306002,-1.125519,-0.960481,-0.697336,0.045282,1.024588,-1.127452,-1.187863,-0.146881,0.0
3,0.005087,0.012214,1.306002,-1.125519,-0.949940,-0.697336,0.045282,1.024588,-1.127452,-1.179098,-0.064682,0.0
4,-0.319737,0.163080,1.541936,-1.115336,-0.939400,-0.448173,0.133498,1.105342,-1.122220,-1.170332,-1.249798,0.0
5,-0.473602,0.146876,2.090254,-1.108083,-0.883485,-0.439868,0.199999,1.029338,-1.112978,-1.123292,-0.292555,0.0
6,0.355556,0.207222,1.540618,-1.101233,-0.912801,0.008626,0.259715,1.122442,-1.104907,-1.130327,-1.053248,0.0
7,0.299139,0.241307,1.636837,-1.086337,-0.811428,0.086697,0.335716,1.174695,-1.086680,-1.032244,-0.317219,1.0
8,0.295720,0.212810,1.539300,-1.084430,-0.900824,0.001982,0.212214,1.165194,-1.076062,-1.109977,0.020956,0.0
9,-0.023976,0.198282,1.432536,-1.067654,-0.739340,0.232873,0.234607,1.155694,-1.062656,-0.948915,-1.145772,0.0


In [39]:
sequential_data, targets = split_dataframe(scaled_df, target_column='Target', window_size=window_size)
reshaped_df = create_sequential_dataframe(sequential_data, targets)
reshaped_df

,AAPL_CALL_OM,AAPL_CALL_O1,AAPL_CALL_OY,AAPL_CALL_OI,AAPL_CALL_VM,AAPL_PUT_OM,AAPL_PUT_O1,AAPL_PUT_OY,AAPL_PUT_OI,AAPL_PUT_VM,Return,Target
0,0 -0.659948 1 0.032441 2 0.005087 3 ...,0 0.023948 1 -0.019636 2 0.012214 3 ...,0 1.232190 1 1.298093 2 1.306002 3 ...,0 -1.135737 1 -1.130776 2 -1.125519 3 ...,0 -0.867662 1 -0.940435 2 -0.960481 3 ...,0 0.066764 1 -0.689031 2 -0.697336 3 ...,0 0.076497 1 0.020175 2 0.045282 3 ...,0 1.069240 1 1.024588 2 1.024588 3 ...,0 -1.141176 1 -1.132042 2 -1.127452 3 ...,0 -1.135861 1 -1.164318 2 -1.187863 3 ...,0 1.939250 1 -0.534837 2 -0.146881 3 ...,0.0
1,0 0.032441 1 0.005087 2 0.005087 3 ...,0 -0.019636 1 0.012214 2 0.012214 3 ...,0 1.298093 1 1.306002 2 1.306002 3 ...,0 -1.130776 1 -1.125519 2 -1.125519 3 ...,0 -0.940435 1 -0.960481 2 -0.949940 3 ...,0 -0.689031 1 -0.697336 2 -0.697336 3 ...,0 0.020175 1 0.045282 2 0.045282 3 ...,0 1.024588 1 1.024588 2 1.024588 3 ...,0 -1.132042 1 -1.127452 2 -1.127452 3 ...,0 -1.164318 1 -1.187863 2 -1.179098 3 ...,0 -0.534837 1 -0.146881 2 -0.064682 3 ...,0.0
2,0 0.005087 1 0.005087 2 -0.319737 3 ...,0 0.012214 1 0.012214 2 0.163080 3 ...,0 1.306002 1 1.306002 2 1.541936 3 ...,0 -1.125519 1 -1.125519 2 -1.115336 3 ...,0 -0.960481 1 -0.949940 2 -0.939400 3 ...,0 -0.697336 1 -0.697336 2 -0.448173 3 ...,0 0.045282 1 0.045282 2 0.133498 3 ...,0 1.024588 1 1.024588 2 1.105342 3 ...,0 -1.127452 1 -1.127452 2 -1.122220 3 ...,0 -1.187863 1 -1.179098 2 -1.170332 3 ...,0 -0.146881 1 -0.064682 2 -1.249798 3 ...,0.0
3,0 0.005087 1 -0.319737 2 -0.473602 3 ...,0 0.012214 1 0.163080 2 0.146876 3 ...,0 1.306002 1 1.541936 2 2.090254 3 ...,0 -1.125519 1 -1.115336 2 -1.108083 3 ...,0 -0.949940 1 -0.939400 2 -0.883485 3 ...,0 -0.697336 1 -0.448173 2 -0.439868 3 ...,0 0.045282 1 0.133498 2 0.199999 3 ...,0 1.024588 1 1.105342 2 1.029338 3 ...,0 -1.127452 1 -1.122220 2 -1.112978 3 ...,0 -1.179098 1 -1.170332 2 -1.123292 3 ...,0 -0.064682 1 -1.249798 2 -0.292555 3 ...,1.0
4,0 -0.319737 1 -0.473602 2 0.355556 3 ...,0 0.163080 1 0.146876 2 0.207222 3 ...,0 1.541936 1 2.090254 2 1.540618 3 ...,0 -1.115336 1 -1.108083 2 -1.101233 3 ...,0 -0.939400 1 -0.883485 2 -0.912801 3 ...,0 -0.448173 1 -0.439868 2 0.008626 3 ...,0 0.133498 1 0.199999 2 0.259715 3 ...,0 1.105342 1 1.029338 2 1.122442 3 ...,0 -1.122220 1 -1.112978 2 -1.104907 3 ...,0 -1.170332 1 -1.123292 2 -1.130327 3 ...,0 -1.249798 1 -0.292555 2 -1.053248 3 ...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3906,0 -0.388121 1 -0.340253 2 -0.533438 3 ...,0 -0.464411 1 -0.515259 2 -0.547108 3 ...,0 -0.644743 1 -0.631562 2 -0.653969 3 ...,0 0.483282 1 0.184886 2 0.209314 3 ...,0 0.155624 1 -0.515012 2 -0.569681 3 ...,0 -0.439868 1 -0.551161 2 -0.414951 3 ...,0 -0.587157 1 -0.646873 2 -0.679445 3 ...,0 -0.622783 1 -0.627533 2 -0.639884 3 ...,0 0.539090 1 0.277769 2 0.290787 3 ...,0 0.266504 1 -0.529382 2 -0.677956 3 ...,0 -0.064682 1 0.202120 2 0.860954 3 ...,1.0
3907,0 -0.340253 1 -0.533438 2 -0.328285 3 ...,0 -0.515259 1 -0.547108 2 -0.452118 3 ...,0 -0.631562 1 -0.653969 2 -0.588066 3 ...,0 0.184886 1 0.209314 2 0.223947 3 ...,0 -0.515012 1 -0.569681 2 -0.610607 3 ...,0 -0.551161 1 -0.414951 2 -0.502989 3 ...,0 -0.646873 1 -0.679445 2 -0.633301 3 ...,0 -0.627533 1 -0.639884 2 -0.566731 3 ...,0 0.277769 1 0.290787 2 0.301707 3 ...,0 -0.529382 1 -0.677956 2 -0.612884 3 ...,0 0.202120 1 0.860954 2 -1.013090 3 ...,1.0
3908,0 -0.533438 1 -0.328285 2 -0.328285 3 ...,0 -0.547108 1 -0.452118 2 -0.483409 3 ...,0 -0.653969 1 -0.588066 2 -0.570931 3 ...,0 0.209314 1 0.223947 2 0.246195 3 ...,0 -0.569681 1 -0.610607 2 -0.429808 3 ...,0 -0.414951 1 -0.502989 2 -0.587705 3 ...,0 -0.679445 1 -0.633301 2 -0.613622 3 ...,0 -0.639884 1 -0.566731 2 -0.659835 3 ...,0 0.290787 1 0.301707 2 0.338419 3 ...,0 -0.677956 1 -0.612884 2 -0.142768 3 ...,0 0.860954 1 -1.013090 2 0.837548 3 ...,0.0
3909,0 -0.328285 1 -0.328285 2 -0.345381 3 ...,0 -0.452118 1 -0.483409 2 -0.435356 3 ...,0 -0.588066 1 -0.570931 2 -0.621018 3 ...,0 0.223947 1 0.24619

In [23]:
np.linspace(0.009, 0.012, 100)

array([0.009     , 0.0090303 , 0.00906061, 0.00909091, 0.00912121,
       0.00915152, 0.00918182, 0.00921212, 0.00924242, 0.00927273,
       0.00930303, 0.00933333, 0.00936364, 0.00939394, 0.00942424,
       0.00945455, 0.00948485, 0.00951515, 0.00954545, 0.00957576,
       0.00960606, 0.00963636, 0.00966667, 0.00969697, 0.00972727,
       0.00975758, 0.00978788, 0.00981818, 0.00984848, 0.00987879,
       0.00990909, 0.00993939, 0.0099697 , 0.01      , 0.0100303 ,
       0.01006061, 0.01009091, 0.01012121, 0.01015152, 0.01018182,
       0.01021212, 0.01024242, 0.01027273, 0.01030303, 0.01033333,
       0.01036364, 0.01039394, 0.01042424, 0.01045455, 0.01048485,
       0.01051515, 0.01054545, 0.01057576, 0.01060606, 0.01063636,
       0.01066667, 0.01069697, 0.01072727, 0.01075758, 0.01078788,
       0.01081818, 0.01084848, 0.01087879, 0.01090909, 0.01093939,
       0.0109697 , 0.011     , 0.0110303 , 0.01106061, 0.01109091,
       0.01112121, 0.01115152, 0.01118182, 0.01121212, 0.01124